In [221]:
from seaborn import load_dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler , OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline , make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

DATA PREPARATION

In [222]:
titanic = load_dataset('titanic')
np.random.seed(42)

In [223]:
titanic

survived  pclass     sex   age  sibsp  parch     fare embarked   class  \
0           0       3    male  22.0      1      0   7.2500        S   Third   
1           1       1  female  38.0      1      0  71.2833        C   First   
2           1       3  female  26.0      0      0   7.9250        S   Third   
3           1       1  female  35.0      1      0  53.1000        S   First   
4           0       3    male  35.0      0      0   8.0500        S   Third   
..        ...     ...     ...   ...    ...    ...      ...      ...     ...   
886         0       2    male  27.0      0      0  13.0000        S  Second   
887         1       1  female  19.0      0      0  30.0000        S   First   
888         0       3  female   NaN      1      2  23.4500        S   Third   
889         1       1    male  26.0      0      0  30.0000        C   First   
890         0       3    male  32.0      0      0   7.7500        Q   Third   

       who  adult_male deck  embark_town alive  alone  
0      man        True  NaN  Southampton    no  False  
1    woman       False    C    Cherbourg   yes  False  
2    woman       False  NaN  Southampton   yes   True  
3    woman       False    C  Southampton   yes  False  
4      man        True  NaN  Southampton    no   True  
..     ...         ...  ...          ...   ...    ...  
886    man        True  NaN  Southampton    no   True  
887  woman       False    B  Southampton   yes   True  
888  woman       False  NaN  Southampton    no  False  
889    man        True    C    Cherbourg   yes   True  
890    man        True  NaN   Queenstown    no   True  

[891 rows x 15 columns]

In [224]:
titanic.isnull().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [225]:
df = pd.DataFrame(data = titanic)

In [226]:
df

survived  pclass     sex   age  sibsp  parch     fare embarked   class  \
0           0       3    male  22.0      1      0   7.2500        S   Third   
1           1       1  female  38.0      1      0  71.2833        C   First   
2           1       3  female  26.0      0      0   7.9250        S   Third   
3           1       1  female  35.0      1      0  53.1000        S   First   
4           0       3    male  35.0      0      0   8.0500        S   Third   
..        ...     ...     ...   ...    ...    ...      ...      ...     ...   
886         0       2    male  27.0      0      0  13.0000        S  Second   
887         1       1  female  19.0      0      0  30.0000        S   First   
888         0       3  female   NaN      1      2  23.4500        S   Third   
889         1       1    male  26.0      0      0  30.0000        C   First   
890         0       3    male  32.0      0      0   7.7500        Q   Third   

       who  adult_male deck  embark_town alive  alone  
0      man        True  NaN  Southampton    no  False  
1    woman       False    C    Cherbourg   yes  False  
2    woman       False  NaN  Southampton   yes   True  
3    woman       False    C  Southampton   yes  False  
4      man        True  NaN  Southampton    no   True  
..     ...         ...  ...          ...   ...    ...  
886    man        True  NaN  Southampton    no   True  
887  woman       False    B  Southampton   yes   True  
888  woman       False  NaN  Southampton    no  False  
889    man        True    C    Cherbourg   yes   True  
890    man        True  NaN   Queenstown    no   True  

[891 rows x 15 columns]

In [227]:
KEEP_FEATURES = ['pclass', 'sex', 'age', 'sibsp', 'embarked']
X = df[KEEP_FEATURES]
y = df[['survived']]

In [228]:
#Columns for pipeline

num_features = ['pclass' , 'age' , 'sibsp']
cat_features = ['sex' , 'embarked']

In [229]:
X_train , X_test ,y_train, y_test = train_test_split(X, y , test_size= 0.2 , random_state= 12)

PIPELINE FOR NUMERICAL AND CATEGROICAL 

In [230]:
#numerical pipeline 
num_pipeline = Pipeline(
    steps=[
        ('impute' , SimpleImputer(strategy= 'mean')),\
        ('scaler' , StandardScaler())
    ]
)

In [231]:
#Categorical pipeline
cat_pipeline = Pipeline(
    steps= [
        ('impute' , SimpleImputer(strategy= 'most_frequent')),
        ('one_hot_encoder' , OneHotEncoder(handle_unknown='ignore' , sparse_output=False))
    ]
)

In [232]:

col_transformer = ColumnTransformer(
    transformers=[
        ('num_cols',num_pipeline , num_features),
        ('cat_cols',cat_pipeline , cat_features)
    ],
    remainder= 'drop'
)

In [233]:
col_transformer

ColumnTransformer(transformers=[('num_cols',
                                 Pipeline(steps=[('impute', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['pclass', 'age', 'sibsp']),
                                ('cat_cols',
                                 Pipeline(steps=[('impute',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('one_hot_encoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 ['sex', 'embarked'])])

In [234]:
pipe_final = Pipeline (
    steps= [
        ('columns_transformer' , col_transformer),
        ('model' , LogisticRegression(max_iter = 100 , random_state = 42))
    ]
)

In [235]:
pipe_final.fit(X_train,y_train)

c:\Users\AMINGO\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('columns_transformer',
                 ColumnTransformer(transformers=[('num_cols',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['pclass', 'age', 'sibsp']),
                                                 ('cat_cols',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one_hot_encoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['sex', 'embarked'])])),
                ('model', LogisticRegression(random_state=42))])

In [236]:
kf = StratifiedKFold(n_splits=10 , shuffle= True , random_state= 12)

In [237]:
cvs = cross_val_score(pipe_final ,X , y.values.ravel() , cv = kf)

In [238]:
cvs

array([0.83333333, 0.84269663, 0.78651685, 0.75280899, 0.79775281,
       0.80898876, 0.74157303, 0.80898876, 0.76404494, 0.7752809 ])

In [239]:
np.average(cvs)

np.float64(0.7911985018726593)